<a href="https://colab.research.google.com/github/spe301/AI-generated-AI/blob/main/DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Potosnail==0.0.7

In [3]:
import pandas as pd
import numpy as np
from google.colab import files
import zipfile
import io
from potosnail import MachineLearning, DeepLearning, DataHelper, Evaluater, Algorithms, Wrappers
from sklearn.datasets import load_breast_cancer, load_boston

#class instantiations
ml = MachineLearning()
dl = DeepLearning()
dh = DataHelper()
ev = Evaluater()
al = Algorithms()
wr = Wrappers()

def BigDawg(X, y, params, task, ts=150, epochs=50, batch_size=32, patience=5, regression=False):
  '''facilitates the Data Collection Process'''
  dl = DeepLearning()
  wr = Wrappers()
  if task == 'NLP':
    X2, y2, _ = wr.TCP(X, y)
    func = dl.RNN #insert CV after this
  if task == 'CV':
    X2, _, y2, _ = dl.ModelReadyPixles(X, y, target_size=(ts, ts))
    func = dl.CNN
  if task == 'TC':
    X2 = np.array(X)
    y2 = dl.MulticlassOutput(np.array(y))
    func = dl.DeepTabularClassification
  if task == 'TR':
    X2 = np.array(X)
    y2 = np.array(y)
    func = dl.DeepTabularRegression
  try:
    df = dl.CollectPerformance(params, func, X2, y2, epochs=epochs, batch_size=batch_size, patience=patience, regression=regression)
  except:
    df = dl.CollectPerformance(params, func, X2, y2.reshape(-1), epochs=epochs, batch_size=batch_size, patience=patience, regression=regression)
  size = []
  n_features = []
  batch = []
  depth = []
  dominances = []
  for i in range(len(df)):
    size.append(X2.shape[0])
    n_features.append(X2.shape[1])
    dominances.append(max(pd.DataFrame(y2).value_counts())/len(y2))
    if task == 'CV':
      depth.append(X2.shape[3])
  df['len_dataset'] = size
  df['n_features'] = n_features
  if regression == False:
    df['dominant_class'] = dominances
  if task == 'CV':
    df['thickness'] = depth
  return df

In [4]:
#Dataframes
Wikipedia = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Wikipedia.csv') #text
Twitter = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Twitter.csv') #text

#Series
articles = Wikipedia['Text']
labels = Wikipedia['AI']
tweets = Twitter['tweet']
sentiments = Twitter['is_there_an_emotion_directed_at_a_brand_or_product']

#arrays
cancer_data = load_breast_cancer()['data']
diagnosis = load_breast_cancer()['target']
housing_data = load_boston()['data']
prices = load_boston()['target']

#images
#uploaded = files.upload()
#data = zipfile.ZipFile(io.BytesIO(uploaded['IMGDS1.zip']), 'r')
#data.extractall()

In [12]:
X = articles
y = labels

df = Wikipedia
data_str = 'Text'
output = len(np.unique(y))
embedding = dh.GetVocab(df, data_str)
rnn_grid = {'output_dim': [output], 'embedding': [embedding], 'nodes': [32, 48, 64], 'activation': ['tanh', 'relu'], 'regularizer': ['L1', None], 
            'stacking': [False, True], 'dropout': [False, True], 'optimizer': ['adam', 'rmsprop'], 'method': ['GRU'], 'bidirectional': [False]}

#output = len(y)
#cnn_grid = {'output_dim': [output], 'base_filters':[32], 'kernel_size': [2], 'activation': ['relu'], 'nblocks': [3], 
            #'pool':[2], 'dropout': [False], 'closer': [False], 'optimizer': ['adam'], 'metrics': ['accuracy']}

#output = len(np.unique(y))
#tc_grid = {'output_dim': [output], 'nodes': [64, 128], 'activation': ['relu'], 'regularizer': [None], 'stacking': [False], 
           #'dropout': [False], 'nlayers': [4], 'closer': [False], 'loss_func': ['binary_crossentropy'], 'optimizer': ['adam']}

#tr_grid = {'nodes': [32, 64, 128], 'activation': ['relu'], 'regularizer': [None], 'stacking': [False], 'dropout': [False], 
           #'nlayers': [4], 'closer': [False], 'loss_func': ['MSE', 'MAE'], 'optimizer': ['rmsprop'], 'y_col': ['string']}

df = BigDawg(X, y, rnn_grid, 'NLP')
df.shape

KeyboardInterrupt: ignored

In [ ]:
df0 = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Text_Classification_Results.csv')

df2 = pd.concat(df0, df)

In [ ]:
df.to_csv('drive/MyDrive/NLP2.csv')

# Image dataset directory graveyard
copy and paste these into a code cell when working with the dataset

Dogs and Cats
train_dir = r'dataset/training_set'

test_dir = r'dataset/test_set'

train_dogs = r'dataset/training_set/dogs'

train_cats = r'dataset/training_set/cats'

test_dogs = r'dataset/test_set/dogs'

test_cats = r'dataset/test_set/cats'

directories = [train_dir, test_dir, train_dogs, train_cats, test_dogs, test_cats]

classes = ['cats', 'dogs']